# Logging with the python standard library

## Intro
* The python standard library comes with a [logging package](https://docs.python.org/3/library/logging.html).
* This standard allows a way for 3rd party modules to share a common API for logging (more on that later).
* Logging is a low-cost and simple way to communicate and record the state of your program.

## Using the basic config
* The most common destination for logs is to write to the stdout stream or to a local file.
* The logging library provides a function for configuring the top-level *root* logger for use in the main python program.

In [1]:
import logging

# Bind the root logger
logger = logging.getLogger(__name__)

# To set up the root logger to send the logs to a file
logging.basicConfig(filename="log.txt", filemode="a", level=logging.INFO) 
logger.info("Logging to a file...") 

# To set up the root logger to send the logs to the console
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True) # force param needed to reconfigure
logger.info("Logging to the stdout stream")
print("abct")

INFO:__main__:Logging to the stdout stream
abct


## Logging Levels
* [Logging levels](https://docs.python.org/3/library/logging.html#logging-levels) have a name and a numeric values to indicate the importance of the logged message.
* When the level parameter is set on a logger, the logger will **ignore** all calls to the logger from a lower logging level.
* When the level is NOTSET, the logger will defer to the higher level logger.

| level | numeric value |
| ----| ----|
| logging.NOTSET | 0 |
| logging.DEBUG | 10 |
| logging.INFO | 20 |
| logging.WARNING | 30 |
| logging.ERROR| 40 |
| logging.CRITICAL | 50 |

* To log a message of a level, you can either use the log method or the build-in method based on the levelname.


In [2]:
logger.log(40, "logging an error message")
logger.error("logging another error message")

ERROR:__main__:logging an error message
ERROR:__main__:logging another error message


## Logging namespaces
* When instantiating a logger. passing the dunder attribute \_\_name\_\_ will preserve the program structure in the output call.
* In the example below, the first function passes a custom string to the function.
    
    ```python
        logger = logging.getLogger("Not using default namespace")
    ```

In [3]:
from custom_loggers import logger_namespace
logger_namespace.create_logger_with_name()
logger_namespace.create_logger_without_name()


INFO:Not using default namespace:This entry is created with a logger with a custom name
INFO:custom_loggers.logger_namespace:This entry is created with a logger without a custom name


## The structure
* The logging structure is made up of a few components: loggers, handlers, filters, and formatters.

### Logger
* Loggers provide the interface to the application code and create the LogRecord objects consumed by the Handlers.
    ```python
        logger = logging.getLogger(__name__)
        logger.info("Log text...")
    ```
* Note that once instantiated, each logger is added to the global namespace. Calling a logger with a name will return the same logger globally. On the upside, loggers do not need to be passed between modules. On the downside, carelessly modifying the same logger may introduce unintended consequences.

In [4]:
logger1 = logging.getLogger("same logger")
logger2 = logging.getLogger("same logger")

logger.info(f"is logger1  also logger2? {logger1 is logger2}")

INFO:__main__:is logger1  also logger2? True


### Handler
* Handlers send the LogRecord to the appropriate destination, some built-in handlers are listed below, but you can also sub-class your own handlers. Each Logger can have multiple Handlers attached.
    * StreamHandler: send messages to streams (file-like objects).
    * FileHandler: send messages to disk files.
    * RotatingFileHandler: send messages to disk files, with support for maximum log file sizes and log file rotation.
    * TimedRotatingFileHandler: send messages to disk files, rotating the log file at certain timed intervals.
    * SocketHandler: send messages to TCP/IP sockets.
    * SMTPHandler: send messages to a designated email address.

* Note that when instantiated, a handler is added to the global _handlerList.

In [5]:
from rich import print as rprint
stderr_handler = logging.StreamHandler(sys.stderr)
logger1.addHandler(stderr_handler)
rprint(logging._handlerList)
rprint(logger1.handlers)


[
    <weakref at 0x000001D6A363D2B0; to 'logging.StreamHandler' at 0x000001D6A3557B60>,
    <weakref at 0x000001D6A366DD50; to 'logging.StreamHandler' at 0x000001D6A1913250>,
    <weakref at 0x000001D6A363DA80; to 'logging.StreamHandler' at 0x000001D6A48FEC10>,
    <weakref at 0x000001D6A4B05530; to 'logging.StreamHandler' at 0x000001D6A4AF0FC0>
]

[<StreamHandler stderr (NOTSET)>]

### Filter
* Filters are attached to Handlers and can be used to filter the LogRecord that are above the logging level for the logger. Multiple Filters can be attached to the same Handler. The record will only be emitted if none of the filters return a false value.
    ```python
        # Filters can also be a function that returns either a LogRecord or a boolean value. 
        #There is no strict need to subclass the base Filter.
        stdout_handler = logging.StreamHandler(sys.stdout)
        stdout_handler.addFilter(lambda x:x.levelno == logging.WARNING)
    ```

In [6]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True) 
logger = logging.getLogger() # use the root logger for this example
for h in logging.getLogger().handlers:
    h.addFilter(lambda record: record.levelno != logging.WARNING) # prevents logging.WARNING level messages from being emitted.
logger.warning("This message will not emit.")
logger.info("This message will emit.")

INFO:root:This message will emit.


### Formatter
* Formatters are used to format the message to the output and various attributes. The logging utility expects the logged object to be a string or have a \_\_str\_\_ method. A full list is available on the [documentation site](https://docs.python.org/3/library/logging.html#logrecord-attributes)

* Some commonly used LogAttribute

| levelname | format |
| --- | ---|
| asctime | \%(asctime)s |
| levelname | \%(levelname)s |
| message | \%(message)s |
| module | \%(module)s |


In [7]:
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="{{%(levelname)s}} %(asctime)s: %(message)s", datefmt="%Y-%m-%d")
handler.setFormatter(formatter)

logging.basicConfig(level=logging.DEBUG, force=True, handlers=[handler]) 
logger.debug("This message uses the specified format")


{{DEBUG}} 2025-06-09: This message uses the specified format


### Thread-safety
* Logging objects use the threading.RLock to ensure thread safety. Use the supplied API for attaching and detaching components.

### Logging Flow
* The official documentation provides [a flow chart](https://docs.python.org/3/howto/logging.html#logging-flow) to show the life of a logging call.

In [8]:
from custom_loggers import SQLiteHandler

sqlite_handler = SQLiteHandler("log.db")
sqlite_handler.setLevel(logging.INFO)
sqlite_handler.addFilter(lambda x: x.levelno == logging.INFO)
std_handler = logging.StreamHandler(sys.stdout)
std_handler.setLevel(logging.DEBUG)
std_handler.addFilter(lambda record: record.levelno == logging.DEBUG)

logging.basicConfig(level=logging.DEBUG, force=True, handlers=(std_handler, sqlite_handler)) 
logger = logging.getLogger()

logger.debug("This message will go to the console")
logger.info("This message will go to a sqlite3 db file")


DEBUG:root:This message will go to the console


An example set up for a logger that only outputs debug messages to the console, sends all info messages to a local sqlite database file.

## Using 3rd party tools
### Rich
The 3rd party richtext library provides a [richtext Handler](https://rich.readthedocs.io/en/stable/logging.html) with color and style support for logging.

In [9]:
from rich.logging import RichHandler
from rich.text import Text
logging.basicConfig(level="INFO", force=True, format = "%(message)s", datefmt="[%X]", handlers=[RichHandler()]
)
logger = logging.getLogger("rich")
logger.info("Add [bold cyan]rich text[/] support with the RichHandler from the [bold red]rich[/] package.", extra={"markup": True})

[09:59:14] INFO     Add rich text support with the RichHandler from the rich package.               ]8;id=220837;file://C:\Users\wwhuang\AppData\Local\Temp\ipykernel_28108\2679340856.py\2679340856.py]8;;\:]8;id=920284;file://C:\Users\wwhuang\AppData\Local\Temp\ipykernel_28108\2679340856.py#6\6]8;;\

### Loguru
[Loguru](https://loguru.readthedocs.io/en/stable/) is a 3rd party library with simple defaults that is compatible with the standard logging library.

In [10]:
from loguru import logger
logger.info("Logging is easy with loguru")


2025-06-09 09:59:14.856 | INFO     | __main__:<module>:2 - Logging is easy with loguru


## Using CLI with flags
* An example is included with the included package.
    * If the environment is installed with pixi:
```bash
pixi run cli_logger # does not show debug logs
pixi run cli_logger --debug # shows debug logs
```


In [11]:
%%bash 
pixi run cli_logger
pixi run cli_logger --debug

[06/09/25 09:59:15] INFO     cli_logger 2025-06-09             cli_logger.py:41
                             09:59:15,601: this is an info log                 
                             record                                            
[06/09/25 09:59:16] INFO     cli_logger 2025-06-09             cli_logger.py:41
                             09:59:16,289: this is an info log                 
                             record                                            
                    DEBUG    cli_logger 2025-06-09             cli_logger.py:42
                             09:59:16,307: this is a debug                     
                             message                                           
